# Auto-Scan Workflow (Microscope + YOLO11)

This notebook walks you through running an automated serpentine scan using your `MicroscopeController` and a YOLO11 model.

### What you'll do here
1. Configure parameters (step sizes, AF policy, LEDs, dedup, paths).
2. Write helper scripts (`auto_scan.py`, `scan_viewer.py`).
3. Connect to the microscope and capture START/END.
4. Run the scan.
5. Optionally launch a quick image viewer.

**Note:** This notebook expects your `MicroscopeController.py` to be importable in the same directory.
If it's elsewhere, adjust `sys.path` accordingly.


In [2]:
# (Optional) If needed, add paths so Python can import MicroscopeController
import sys, os
sys.path.append(os.getcwd())  # current folder
print('Working dir:', os.getcwd())
print('Python:', sys.version)


Working dir: d:\Install\HQ_Transfer\Manuals\Programming examples\Commandserver\Python
Python: 3.10.18 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:08:55) [MSC v.1929 64 bit (AMD64)]


## 1) Install dependencies (run locally)
If you don't already have them:
```
pip install ultralytics numpy pandas scipy flask
```
You can run that in a terminal or uncomment the next cell to try via pip here.

In [ ]:
# !pip install ultralytics numpy pandas scipy flask

## 2) Configure your scan
Edit values below to your liking.


In [ ]:
from datetime import datetime

# --- Model & calibration ---
WEIGHTS = r"E:\\0_FlakesNew\\runs\\train\\AI_BN_1002\\weights\\best.pt"  # YOLO11 .pt path
CALIB_TXT = r"D:\Install\HQ_Transfer\Manuals\Programming examples\Commandserver\Python\HQ_pixel_size.txt"  # Calibration text file (um/pixel values)

# --- Scan geometry --- (mm; stage units are mm)
STEP_X_MM = 0.20
STEP_Y_MM = 0.20
SETTLE_MS = 50
SAFE_Z = None   # set a safe Z (e.g., 0.0) or leave as None to disable lift-before-move

# --- AF policy ---
AF_10X_PER_TILE = True
AF_50X = True
AF_DISTANCE_10X = 0.01
AF_DISTANCE_50X = 0.01

# --- Lighting ---
LED_PERCENT = 15

# --- Detection ---
CONF = 0.25
IOU  = 0.45
ALLOW_CLASSES = None  # e.g., ['thin','mid','thick'] or None for all

# --- Candidate handling ---
DEDUP_RADIUS_MM = 0.10     # adjust to taste
CENTER_TOL_MM   = 0.015    # between 0.01 and 0.02 per your preference
MAX_CENTER_ITERS = 3

# --- Output ---
OUT_ROOT  = r"E:\0_FlakesVideo\scans"  # default will be E:\Scans\{date}\{scan_name}
SCAN_NAME = None  # default will be scan_{timestamp}

# --- Image→stage mapping extras ---
FLIP_X = False
FLIP_Y = False
ROTATE_DEG = 0.0

print('Config loaded.')


## 3) Write helper scripts to disk
This writes `auto_scan.py` and `scan_viewer.py` into the working folder.


In [ ]:
auto_scan_code = r'''from __future__ import annotations
import argparse
import json
import math
import os
import sys
import time
from dataclasses import dataclass
from datetime import datetime
from typing import List, Tuple, Dict, Any, Optional

import numpy as np
import pandas as pd
from scipy.spatial import cKDTree as KDTree

try:
    from ultralytics import YOLO
except Exception as e:
    YOLO = None

from MicroscopeController import MicroscopeController

def load_um_per_px_from_txt(txt_path: str) -> Dict[str, float]:
    d: Dict[str, float] = {}
    if not os.path.exists(txt_path):
        return d
    with open(txt_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line or line.lower().endswith('um/pixel'):
                continue
            if ':' in line:
                key, val = line.split(':', 1)
                key = key.strip().upper()
                try:
                    d[key] = float(val.strip())
                except ValueError:
                    pass
    return d

@dataclass
class CameraMapping:
    px_per_mm_10x: float
    px_per_mm_50x: float
    flip_x: bool = False
    flip_y: bool = False
    rotate_deg: float = 0.0
    def img_dxdy_px_to_stage_mm(self, dx_px: float, dy_px: float, use_50x: bool = False) -> Tuple[float, float]:
        px_per_mm = self.px_per_mm_50x if use_50x else self.px_per_mm_10x
        dx_mm = dx_px / px_per_mm
        dy_mm = dy_px / px_per_mm
        if self.flip_x:
            dx_mm = -dx_mm
        if self.flip_y:
            dy_mm = -dy_mm
        if abs(self.rotate_deg) > 1e-9:
            import math
            th = math.radians(self.rotate_deg)
            cos_t, sin_t = math.cos(th), math.sin(th)
            sx = dx_mm * cos_t - dy_mm * sin_t
            sy = dx_mm * sin_t + dy_mm * cos_t
            return sx, sy
        return dx_mm, dy_mm

@dataclass
class ScanConfig:
    step_x_mm: float = 0.20
    step_y_mm: float = 0.20
    settle_ms: int = 50
    safe_z: Optional[float] = None
    led_percent: int = 15
    af_10x_per_tile: bool = True
    af_50x: bool = True
    af_distance_10x: float = 0.01
    af_distance_50x: float = 0.01
    yolo_weights: str = r"E:\\0_FlakesNew\\runs\\train\\AI_BN_1002\\weights\\best.pt"
    conf_threshold: float = 0.25
    iou_threshold: float = 0.45
    allow_classes: Optional[List[str]] = None
    dedup_radius_mm: float = 0.10
    center_tol_mm: float = 0.015
    max_center_iters: int = 3
    root_out: Optional[str] = None
    scan_name: Optional[str] = None
    calib_txt: Optional[str] = r"/mnt/data/HQ_pixel_size.txt"
    flip_x: bool = False
    flip_y: bool = False
    rotate_deg: float = 0.0

class AutoScanner:
    def __init__(self, mc: MicroscopeController, cfg: ScanConfig):
        import math
        self.mc = mc
        self.cfg = cfg
        self.mapping = self._build_mapping()
        self.model = self._load_model()
        self.dedup_tree = None
        self.candidate_pts: List[Tuple[float, float]] = []
        self.results_csv_rows: List[Dict[str, Any]] = []
        self.results_json: Dict[str, Any] = {"tiles": [], "candidates": []}

    def _build_mapping(self) -> CameraMapping:
        um_px = load_um_per_px_from_txt(self.cfg.calib_txt) if self.cfg.calib_txt else {}
        def px_per_mm_for(mag_key: str, fallback_um_per_px: float) -> float:
            um_per_px = um_px.get(mag_key, fallback_um_per_px)
            if um_per_px <= 0:
                raise ValueError(f"Invalid um/px for {mag_key}")
            return 1000.0 / um_per_px
        px_per_mm_10x = px_per_mm_for("10X", 0.35)
        px_per_mm_50x = px_per_mm_for("50X", 0.064)
        return CameraMapping(px_per_mm_10x, px_per_mm_50x, self.cfg.flip_x, self.cfg.flip_y, self.cfg.rotate_deg)

    def _load_model(self):
        if YOLO is None:
            raise RuntimeError("Ultralytics not available. Install with `pip install ultralytics`.")
        return YOLO(self.cfg.yolo_weights)

    def _update_dedup_tree(self):
        if self.candidate_pts:
            self.dedup_tree = KDTree(np.array(self.candidate_pts))
        else:
            self.dedup_tree = None

    def _is_far_from_existing(self, x_mm: float, y_mm: float) -> bool:
        if self.dedup_tree is None:
            return True
        d, _ = self.dedup_tree.query([x_mm, y_mm], k=1)
        return d >= self.cfg.dedup_radius_mm

    def _maybe_lift_safe_z(self):
        if self.cfg.safe_z is not None:
            self.mc.set_z_position(self.cfg.safe_z)

    def _settle(self):
        time.sleep(self.cfg.settle_ms / 1000.0)

    def _goto_xy(self, x_mm: float, y_mm: float):
        self._maybe_lift_safe_z()
        self.mc.set_x_position(x_mm)
        self.mc.set_y_position(y_mm)
        self._settle()

    def _autofocus(self, distance: float) -> bool:
        ok = bool(self.mc.autofocus(distance))
        if not ok:
            time.sleep(0.1)
            ok = bool(self.mc.autofocus(distance))
        return ok

    def _take_picture(self, path: str) -> bool:
        os.makedirs(os.path.dirname(path), exist_ok=True)
        return bool(self.mc.take_picture(path))

    def run_detection_on_image(self, img_path: str) -> List[Dict[str, Any]]:
        res = self.model.predict(img_path, conf=self.cfg.conf_threshold, iou=self.cfg.iou_threshold, verbose=False)
        out: List[Dict[str, Any]] = []
        if not res:
            return out
        r0 = res[0]
        names = r0.names
        if r0.boxes is None:
            return out
        boxes = r0.boxes.xyxy.cpu().numpy()
        confs = r0.boxes.conf.cpu().numpy()
        clss = r0.boxes.cls.cpu().numpy().astype(int)
        h, w = r0.orig_shape
        cx_img = w / 2.0
        cy_img = h / 2.0
        for (x1, y1, x2, y2), c, ci in zip(boxes, confs, clss):
            name = names.get(int(ci), str(ci))
            if self.cfg.allow_classes and name not in self.cfg.allow_classes:
                continue
            cx = (x1 + x2) / 2.0
            cy = (y1 + y2) / 2.0
            out.append({
                "bbox": [float(x1), float(y1), float(x2), float(y2)],
                "conf": float(c),
                "cls": int(ci),
                "cls_name": name,
                "cx": float(cx),
                "cy": float(cy),
                "img_center": [cx_img, cy_img],
                "img_size": [w, h],
            })
        return out

    def center_candidate(self, det: Dict[str, Any], current_xy_mm: Tuple[float, float]) -> Tuple[float, float, bool]:
        x_mm, y_mm = current_xy_mm
        for _ in range(self.cfg.max_center_iters):
            cx, cy = det["cx"], det["cy"]
            cx_img, cy_img = det["img_center"]
            dx_px = cx - cx_img
            dy_px = cy - cy_img
            dx_mm, dy_mm = self.mapping.img_dxdy_px_to_stage_mm(dx_px, dy_px, use_50x=False)
            x_mm_new = x_mm + dx_mm
            y_mm_new = y_mm + dy_mm
            self._goto_xy(x_mm_new, y_mm_new)
            x_mm, y_mm = x_mm_new, y_mm_new
            if (dx_mm**2 + dy_mm**2) ** 0.5 <= self.cfg.center_tol_mm:
                return x_mm, y_mm, True
            tmp_path = os.path.join(self.out_dir, "tmp", f"recenter_{datetime.now().strftime('%Y%m%d_%H%M%S')}.jpg")
            self._take_picture(tmp_path)
            dets = self.run_detection_on_image(tmp_path)
            if not dets:
                break
            det = sorted(dets, key=lambda d: (d["cx"]-cx_img)**2 + (d["cy"]-cy_img)**2)[0]
        return x_mm, y_mm, False

    def run_scan(self, start_xy: Tuple[float, float], end_xy: Tuple[float, float]):
        date_str = datetime.now().strftime('%Y%m%d')
        scan_name = self.cfg.scan_name or f"scan_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        root = self.cfg.root_out or os.path.join("E:\\Scans", date_str, scan_name)
        self.out_dir = root
        tiles_dir = os.path.join(root, "tiles_10x")
        cand_dir = os.path.join(root, "candidates_50x")
        os.makedirs(tiles_dir, exist_ok=True)
        os.makedirs(cand_dir, exist_ok=True)
        os.makedirs(os.path.join(root, "tmp"), exist_ok=True)

        try:
            self.mc.set_brightness(self.cfg.led_percent)
        except Exception:
            pass
        self.mc.set_magnification(10)

        x0, y0 = start_xy
        x1, y1 = end_xy
        x_min, x_max = sorted([x0, x1])
        y_min, y_max = sorted([y0, y1])
        n_cols = max(1, int(round((x_max - x_min) / self.cfg.step_x_mm)) + 1)
        n_rows = max(1, int(round((y_max - y_min) / self.cfg.step_y_mm)) + 1)
        self.results_json["meta"] = {
            "start_xy": [x0, y0],
            "end_xy": [x1, y1],
            "x_range": [x_min, x_max],
            "y_range": [y_min, y_max],
            "n_cols": n_cols,
            "n_rows": n_rows,
            "step_x_mm": self.cfg.step_x_mm,
            "step_y_mm": self.cfg.step_y_mm,
            "led_percent": self.cfg.led_percent,
            "af_10x_per_tile": self.cfg.af_10x_per_tile,
            "af_50x": self.cfg.af_50x,
            "dedup_radius_mm": self.cfg.dedup_radius_mm,
            "center_tol_mm": self.cfg.center_tol_mm,
        }
        xs = [x_min + c * self.cfg.step_x_mm for c in range(n_cols)]
        ys = [y_min + r * self.cfg.step_y_mm for r in range(n_rows)]

        for ci, x in enumerate(xs):
            col_ys = ys if (ci % 2 == 0) else list(reversed(ys))
            for ri, y in enumerate(col_ys):
                self._goto_xy(x, y)
                if self.cfg.af_10x_per_tile:
                    self._autofocus(self.cfg.af_distance_10x)
                tile_name = f"tile_x{ci:03d}_y{ri:03d}_X{x:.3f}_Y{y:.3f}.jpg"
                tile_path = os.path.join(tiles_dir, tile_name)
                if not self._take_picture(tile_path):
                    raise RuntimeError(f"Failed to capture 10x tile at X={x:.3f}, Y={y:.3f}")
                dets = self.run_detection_on_image(tile_path)
                self.results_json["tiles"].append({"ci": ci, "ri": ri, "X": x, "Y": y, "path": tile_path, "detections": dets})
                for det in dets:
                    cur_x, cur_y = x, y
                    if self._is_far_from_existing(cur_x, cur_y):
                        new_x, new_y, centered_ok = self.center_candidate(det, (cur_x, cur_y))
                        if centered_ok and self._is_far_from_existing(new_x, new_y):
                            self.candidate_pts.append((new_x, new_y))
                            self._update_dedup_tree()
                            self.mc.set_magnification(50)
                            if self.cfg.af_50x:
                                self._autofocus(self.cfg.af_distance_50x)
                            cand_name = f"cand_X{new_x:.3f}_Y{new_y:.3f}_cls{det['cls_name']}_conf{det['conf']:.2f}.jpg"
                            cand_path = os.path.join(cand_dir, cand_name)
                            ok2 = self._take_picture(cand_path)
                            self.results_json["candidates"].append({"X": new_x, "Y": new_y, "path": cand_path if ok2 else None, "det": det, "centered": centered_ok})
                            self.results_csv_rows.append({
                                "type": "candidate_50x", "X_mm": new_x, "Y_mm": new_y,
                                "tile_x_index": ci, "tile_y_index": ri,
                                "tile_path": tile_path, "cand_path": cand_path if ok2 else "",
                                "cls": det["cls_name"], "conf": det["conf"],
                            })
                            self.mc.set_magnification(10)
                        else:
                            self.results_csv_rows.append({
                                "type": "candidate_skip", "X_mm": new_x, "Y_mm": new_y,
                                "tile_x_index": ci, "tile_y_index": ri,
                                "tile_path": tile_path, "reason": "not_centered_or_duplicate",
                            })
        csv_path = os.path.join(root, "scan_log.csv")
        json_path = os.path.join(root, "scan_log.json")
        pd.DataFrame(self.results_csv_rows).to_csv(csv_path, index=False)
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(self.results_json, f, indent=2)
        print(f"Saved logs: {csv_path}\n{json_path}")

def parse_args():
    p = argparse.ArgumentParser(description="Microscope auto-scan with YOLO11")
    p.add_argument('--start_xy', type=float, nargs=2)
    p.add_argument('--end_xy', type=float, nargs=2)
    p.add_argument('--step_x_mm', type=float, default=0.20)
    p.add_argument('--step_y_mm', type=float, default=0.20)
    p.add_argument('--settle_ms', type=int, default=50)
    p.add_argument('--safe_z', type=float, default=None)
    p.add_argument('--af_10x_per_tile', action='store_true', default=True)
    p.add_argument('--no_af_10x_per_tile', dest='af_10x_per_tile', action='store_false')
    p.add_argument('--af_50x', action='store_true', default=True)
    p.add_argument('--no_af_50x', dest='af_50x', action='store_false')
    p.add_argument('--af_distance_10x', type=float, default=0.01)
    p.add_argument('--af_distance_50x', type=float, default=0.01)
    p.add_argument('--led_percent', type=int, default=15)
    p.add_argument('--weights', type=str, default=r"E:\\0_FlakesNew\\runs\\train\\AI_BN_1002\\weights\\best.pt")
    p.add_argument('--conf', type=float, default=0.25)
    p.add_argument('--iou', type=float, default=0.45)
    p.add_argument('--allow_classes', type=str, nargs='*', default=None)
    p.add_argument('--dedup_radius_mm', type=float, default=0.10)
    p.add_argument('--center_tol_mm', type=float, default=0.015)
    p.add_argument('--max_center_iters', type=int, default=3)
    p.add_argument('--out_root', type=str, default=None)
    p.add_argument('--scan_name', type=str, default=None)
    p.add_argument('--calib_txt', type=str, default=r"/mnt/data/HQ_pixel_size.txt")
    p.add_argument('--flip_x', action='store_true', default=False)
    p.add_argument('--flip_y', action='store_true', default=False)
    p.add_argument('--rotate_deg', type=float, default=0.0)
    return p.parse_args()

def acquire_start_end(mc: MicroscopeController, args) -> Tuple[Tuple[float, float], Tuple[float, float]]:
    if args.start_xy is not None:
        start_xy = (args.start_xy[0], args.start_xy[1])
    else:
        x = float(mc.get_x_position()); y = float(mc.get_y_position())
        print(f"Captured START: X={x:.3f}, Y={y:.3f}")
        start_xy = (x, y)
    if args.end_xy is not None:
        end_xy = (args.end_xy[0], args.end_xy[1])
    else:
        input("Move stage to END, then press <Enter>...")
        x = float(mc.get_x_position()); y = float(mc.get_y_position())
        print(f"Captured END: X={x:.3f}, Y={y:.3f}")
        end_xy = (x, y)
    return start_xy, end_xy

def main():
    args = parse_args()
    cfg = ScanConfig(
        step_x_mm=args.step_x_mm, step_y_mm=args.step_y_mm,
        settle_ms=args.settle_ms, safe_z=args.safe_z,
        led_percent=args.led_percent,
        af_10x_per_tile=args.af_10x_per_tile, af_50x=args.af_50x,
        af_distance_10x=args.af_distance_10x, af_distance_50x=args.af_distance_50x,
        yolo_weights=args.weights, conf_threshold=args.conf, iou_threshold=args.iou,
        allow_classes=args.allow_classes,
        dedup_radius_mm=args.dedup_radius_mm, center_tol_mm=args.center_tol_mm, max_center_iters=args.max_center_iters,
        root_out=args.out_root, scan_name=args.scan_name,
        calib_txt=args.calib_txt, flip_x=args.flip_x, flip_y=args.flip_y, rotate_deg=args.rotate_deg,
    )
    mc = MicroscopeController()
    try:
        start_xy, end_xy = acquire_start_end(mc, args)
        scanner = AutoScanner(mc, cfg)
        scanner.run_scan(start_xy, end_xy)
        print("Scan completed.")
    except KeyboardInterrupt:
        print("Interrupted by user.")
    except Exception as e:
        print(f"FATAL: {e}\nAborting.")
    finally:
        try:
            mc.led_off()
        except Exception:
            pass
        mc.close()

if __name__ == '__main__':
    main()
'''

scan_viewer_code = r'''from __future__ import annotations
import argparse, json, os
from flask import Flask, render_template_string, send_from_directory
TEMPLATE = """
<!doctype html><html><head><meta charset='utf-8'/><title>Scan Viewer</title>
<style>body{font-family:Arial;margin:1rem 2rem}.grid{display:grid;grid-template-columns:repeat(auto-fill,minmax(260px,1fr));gap:12px}.card{border:1px solid #ccc;border-radius:8px;padding:10px}img{max-width:100%;height:auto;border:1px solid #ddd;border-radius:6px}.path{font-size:.85em;color:#555;word-break:break-all}.meta{margin-bottom:1rem}.badge{display:inline-block;padding:2px 6px;border:1px solid #666;border-radius:6px;font-size:.8em}</style>
</head><body>
<h2>Scan Viewer</h2>
<div class='meta'><div class='badge'>{{meta.n_cols}} cols × {{meta.n_rows}} rows</div> <div class='badge'>step: {{meta.step_x_mm}} × {{meta.step_y_mm}} mm</div> <div class='badge'>LED: {{meta.led_percent}}%</div></div>
<h3>Tiles (10×)</h3>
<div class='grid'>{% for t in tiles %}<div class='card'><div><strong>Tile:</strong> x{{t.ci}} y{{t.ri}} @ ({{'%.3f'|format(t.X)}}, {{'%.3f'|format(t.Y)}})</div><a href='/img?path={{t.path}}'><img src='/img?path={{t.path}}'/></a><div class='path'>{{t.path}}</div>{% if t.detections and t.detections|length > 0 %}<div><strong>Detections:</strong> {{t.detections|length}}</div><ul>{% for d in t.detections %}<li>{{d.cls_name}} ({{'%.2f'|format(d.conf)}})</li>{% endfor %}</ul>{% else %}<div>No detections</div>{% endif %}</div>{% endfor %}</div>
<h3>Candidates (50×)</h3>
<div class='grid'>{% for c in cands %}<div class='card'><div><strong>Candidate @ ({{'%.3f'|format(c.X)}}, {{'%.3f'|format(c.Y)}})</strong></div>{% if c.path %}<a href='/img?path={{c.path}}'><img src='/img?path={{c.path}}'/></a><div class='path'>{{c.path}}</div>{% else %}<div>Image missing</div>{% endif %}{% if c.det %}<div>Det: {{c.det.cls_name}} ({{'%.2f'|format(c.det.conf)}})</div>{% endif %}</div>{% endfor %}</div>
</body></html>"""
app = Flask(__name__)
@app.route('/img')
def img():
    from flask import request, abort
    path = request.args.get('path')
    if not path or not os.path.exists(path):
        abort(404)
    return send_from_directory(os.path.dirname(path), os.path.basename(path))
@app.route('/')
def index():
    log_path = app.config['LOG_PATH']
    with open(log_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    meta = data.get('meta', {})
    tiles = data.get('tiles', [])
    cands = data.get('candidates', [])
    from flask import render_template_string
    return render_template_string(TEMPLATE, meta=meta, tiles=tiles, cands=cands)
def main():
    ap = argparse.ArgumentParser()
    ap.add_argument('--root', required=True)
    ap.add_argument('--port', type=int, default=5000)
    args = ap.parse_args()
    log_path = os.path.join(args.root, 'scan_log.json')
    if not os.path.exists(log_path):
        raise SystemExit(f'scan_log.json not found under: {args.root}')
    app.config['LOG_PATH'] = log_path
    app.run(host='127.0.0.1', port=args.port, debug=False)
if __name__ == '__main__':
    main()
'''

with open('auto_scan.py', 'w', encoding='utf-8') as f:
    f.write(auto_scan_code)
with open('scan_viewer.py', 'w', encoding='utf-8') as f:
    f.write(scan_viewer_code)
print('Wrote auto_scan.py and scan_viewer.py to', os.getcwd())


## 4) Connect to MicroscopeController
This just imports your controller and creates a connection (handled internally by the class).

In [ ]:
from MicroscopeController import MicroscopeController
mc = MicroscopeController()
print('Connected.')
# Initialize LED
try:
    mc.set_brightness(LED_PERCENT)
except Exception as e:
    print('LED brightness set failed (will continue):', e)


## 5) Capture START and END
If you set `START_XY` / `END_XY` manually, you can skip the prompts. Otherwise:
- START is read from your *current* stage position.
- Then you'll move to END and press Enter to capture.


In [ ]:
START_XY = None  # e.g., (10.0, 15.0)
END_XY   = None  # e.g., (12.0, 17.0)

def capture_start_end(mc):
    import time
    if START_XY is not None:
        start_xy = START_XY
    else:
        x = float(mc.get_x_position()); y = float(mc.get_y_position())
        print(f"Captured START from live stage: X={x:.3f}, Y={y:.3f}")
        start_xy = (x, y)
    if END_XY is not None:
        end_xy = END_XY
    else:
        input('Move stage to END point, then press <Enter>...')
        x = float(mc.get_x_position()); y = float(mc.get_y_position())
        print(f"Captured END from live stage: X={x:.3f}, Y={y:.3f}")
        end_xy = (x, y)
    return start_xy, end_xy

start_xy, end_xy = capture_start_end(mc)
start_xy, end_xy


## 6) Run the scan
This cell builds a configuration from your settings and kicks off the serpentine scan.


In [ ]:
from auto_scan import ScanConfig, AutoScanner

cfg = ScanConfig(
    step_x_mm=STEP_X_MM,
    step_y_mm=STEP_Y_MM,
    settle_ms=SETTLE_MS,
    safe_z=SAFE_Z,
    led_percent=LED_PERCENT,
    af_10x_per_tile=AF_10X_PER_TILE,
    af_50x=AF_50X,
    af_distance_10x=AF_DISTANCE_10X,
    af_distance_50x=AF_DISTANCE_50X,
    yolo_weights=WEIGHTS,
    conf_threshold=CONF,
    iou_threshold=IOU,
    allow_classes=ALLOW_CLASSES,
    dedup_radius_mm=DEDUP_RADIUS_MM,
    center_tol_mm=CENTER_TOL_MM,
    max_center_iters=MAX_CENTER_ITERS,
    root_out=OUT_ROOT,
    scan_name=SCAN_NAME,
    calib_txt=CALIB_TXT,
    flip_x=FLIP_X,
    flip_y=FLIP_Y,
    rotate_deg=ROTATE_DEG,
)

scanner = AutoScanner(mc, cfg)
scanner.run_scan(start_xy, end_xy)
print('Done.')


## 7) Peek at logs
The scan writes a CSV and a JSON log under the output root. This cell prints where they are.

In [ ]:
import os, json
root = scanner.out_dir
print('Scan root:', root)
print('CSV :', os.path.join(root, 'scan_log.csv'))
print('JSON:', os.path.join(root, 'scan_log.json'))
with open(os.path.join(root, 'scan_log.json'), 'r', encoding='utf-8') as f:
    data = json.load(f)
print('Tiles logged:', len(data.get('tiles', [])))
print('Candidates logged:', len(data.get('candidates', [])))


## 8) Launch the viewer (optional)
This starts a small local Flask app. Run after the scan completes.

In a separate terminal, you could also run:
```
python scan_viewer.py --root <scan_root>
```


In [ ]:
# Example (uncomment to run):
# import subprocess
# subprocess.Popen([sys.executable, 'scan_viewer.py', '--root', scanner.out_dir])
# print('Viewer started. Open http://127.0.0.1:5000 in your browser.')


## 9) Clean up
Turn off LED and close the controller when you’re done.

In [ ]:
try:
    mc.led_off()
except Exception:
    pass
mc.close()
print('Controller closed.')
